"Sentence Features for Extractive Summarization:
1. Surface Features: Based on structure of documents or sentences, including **sentence position** in the document, the **number of words in the sentence**, and the number of quoted words in the sentence.
2. Content Features: Integrated three well-known sentence features based on content-bearing words i.e., centroid words, **signature terms**, and **high frequency words**.
3. Event Features: An **event** is comprised of an event term and associated event elements.
4. Relevance features: Incorporated to exploit inter-sentence relationships. It is assumed that: (1) sentences related to important sentences are important; (2) sentences related to many other sentences are important. The first sentence in a document or a paragraph is important, and other sentences in a document are compared with the leading ones."

(We have chosen to use the features in bold.)

-- *Extractive Summarization Using Supervised and Semi-Supervised Learning*

http://anthology.aclweb.org/C/C08/C08-1124.pdf

In [1]:
from collections import Counter
import pandas as pd
import spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 326.8 kB/s eta 0:00:40
     --------------------------------------- 0.1/12.8 MB 525.1 kB/s eta 0:00:25
      --------------------------------------- 0.3/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.6/12.8 MB 2.3 MB/s eta 0:00:06
     --- ------------------------------------ 1.1/12.8 MB 3.9 MB/s eta 0:00:04
     ----- ---------------------------------- 1.7/12.8 MB 5.3 MB/s eta 0:00:03
     ------- -------------------------------- 2.3/12.8 MB 6.0 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.8 MB 6.8 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.8 MB 7.4 MB/s eta 0:00:02
     ------------ --------------------------- 3.9/12.8 MB 7.8 MB/s eta 

In [2]:
# Load the processed and cleaned small portion of dataframes.
train_one_percent = pd.read_csv("C:/Users/ankar/Downloads/NLU/drive/processed-cleaned_validation.csv")
val_one_percent = pd.read_csv("C:/Users/ankar/Downloads/NLU/drive/processed-cleaned_validation.csv")
test_one_percent = pd.read_csv("C:/Users/ankar/Downloads/NLU/drive/processed-cleaned_test.csv")

In [3]:
# Hold the three datasets.
datasets = {'train': train_one_percent, 'validation': val_one_percent, 'test': test_one_percent}

In [4]:
datasets['train'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   article             134 non-null    object
 1   highlights          134 non-null    object
 2   id                  134 non-null    object
 3   internet-free_art   134 non-null    object
 4   internet-free_high  134 non-null    object
 5   boiler-free_art     134 non-null    object
 6   boiler-free_high    134 non-null    object
 7   clean_sents_art     134 non-null    object
 8   clean_sents_high    134 non-null    object
dtypes: object(9)
memory usage: 9.6+ KB


In [5]:
datasets['validation'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   article             134 non-null    object
 1   highlights          134 non-null    object
 2   id                  134 non-null    object
 3   internet-free_art   134 non-null    object
 4   internet-free_high  134 non-null    object
 5   boiler-free_art     134 non-null    object
 6   boiler-free_high    134 non-null    object
 7   clean_sents_art     134 non-null    object
 8   clean_sents_high    134 non-null    object
dtypes: object(9)
memory usage: 9.6+ KB


In [6]:
datasets['test'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   article             115 non-null    object
 1   highlights          115 non-null    object
 2   id                  115 non-null    object
 3   internet-free_art   115 non-null    object
 4   internet-free_high  115 non-null    object
 5   boiler-free_art     115 non-null    object
 6   boiler-free_high    115 non-null    object
 7   clean_sents_art     115 non-null    object
 8   clean_sents_high    115 non-null    object
dtypes: object(9)
memory usage: 8.2+ KB


In [7]:
datasets['validation'].head(3)

,article,highlights,id,internet-free_art,internet-free_high,boiler-free_art,boiler-free_high,clean_sents_art,clean_sents_high
0,Since giving birth to her daughter Arabella fo...,Critics have made cruel taunts about Rebecca F...,911d354c3ae2eefdd0b3e2e4de98f35205da1fe4,Since giving birth to her daughter Arabella fo...,Critics have made cruel taunts about Rebecca F...,Since giving birth to her daughter Arabella fo...,Critics have made cruel taunts about Rebecca F...,"[['give', 'birth', 'daughter', 'arabella', 'mo...","[['critic', 'cruel', 'taunt', 'rebecca', 'ferg..."
1,Ireland clung on to claim a controversial five...,Ireland beat Zimbabwe by five runs after excit...,47aea873200747c753d8eea82b6fd8c622324104,Ireland clung on to claim a controversial five...,Ireland beat Zimbabwe by five runs after excit...,Ireland clung on to claim a controversial five...,Ireland beat Zimbabwe by five runs after excit...,"[['ireland', 'clung', 'claim', 'controversial'...","[['ireland', 'beat', 'zimbabwe', 'run', 'excit..."
2,Laurence Fishburne is a multimillionaire actor...,Hattie Crawford Fishburne says she will be evi...,59d85872c0649cc425ae34dfcf35e4a0044575b3,Laurence Fishburne is a multimillionaire actor...,Hattie Crawford Fishburne says she will be evi...,Laurence Fishburne is a multimillionaire actor...,Hattie Crawford Fishburne says she will be evi...,"[['laurence', 'fishburne', 'multimillionaire',...","[['hattie', 'crawford', 'fishburne', 'say', 'e..."


In [8]:
# For readability.
# pd.set_option('display.max_colwidth', 200)
pd.set_option('max_colwidth', None)
# pd.set_option('display.max_columns', None)

## Sentence Embeddings
For calculating word embeddings, it's generally more effective to use the original tokens rather than lemmatized ones. This is because word embeddings are typically trained on large corpora of text in their original form, and using lemmatized tokens might lead to a loss of semantic information that's crucial for accurate embeddings.

To calculate word embeddings using the Gensim library, you can use a pre-trained model like Word2Vec, GloVe, or FastText. Gensim provides easy access to these models.

When deciding whether to use the cleaner version of your articles and highlights (without boilerplate and internet clutter) or the original version for creating sentence embeddings, there are a few considerations to keep in mind:

1. Relevance and Noise: The cleaner versions of your articles and highlights are likely to be more relevant and less noisy. Removing elements like boilerplate text, HTML tags, and URLs helps focus on the actual content. For tasks like summarization, where the quality and relevance of the text are crucial, using cleaner data can lead to more accurate and meaningful embeddings.

2. Context Preservation: If the process of cleaning does not significantly alter or remove important contextual information from the articles and highlights, then using the cleaner versions is preferable. However, if the cleaning process might strip away context or alter the meaning of the text, the original version should be considered.

3. Computational Efficiency: The cleaner versions are likely more concise, which can make the processing for embedding generation more efficient. Less irrelevant data (like HTML tags) means quicker processing and potentially better embeddings.

4. Alignment with Task Objectives: Consider what aligns best with the objectives of your summarization task. If the goal is to generate summaries that are clean and free of such clutter, then using the cleaner versions for training makes sense.

Given these considerations, for most scenarios, using the cleaner version of the articles and highlights would be the better choice. It allows you to focus on the substantive content of the text, leading to more accurate and meaningful sentence embeddings. This is especially true for summarization tasks, where the quality of the content is more important than the original formatting or extraneous text.

In [9]:
!pip install gensim

In [32]:
import gensim.downloader as api
import numpy as np

# Load pre-trained Word2Vec model from Gensim.
model = api.load("word2vec-google-news-300")

# For the non-aggressive sentence embeddings on 'boiler-free' text:
#nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])  # Disable for speed.
#nlp.add_pipe('sentencizer')  # Add sentencizer to the pipeline for extra help in sentence tokenization.


def get_sentence_embeddings(lemmatized_sentences):
    sentence_embeddings = []

    for sentence in lemmatized_sentences:
        # Filter words present in the Word2Vec model's vocabulary
        words = [word for word in sentence if word in model.key_to_index]

        if words:
            embeddings = [model[word] for word in words]
            mean_embedding = np.mean(embeddings, axis=0)
            sentence_embeddings.append(mean_embedding)
        else:
            sentence_embeddings.append(np.zeros(model.vector_size))
    # Returns a list of embedding vectors, one for each sentence.
    return sentence_embeddings

import ast

# Function to convert string representation of list to actual list.
def string_to_list(text):
    try:
        return ast.literal_eval(text)
    except Exception as e:
        return []


In [44]:
# Example 3
txt = """[['blackpool', 'pier', 'face', 'uncertain', 'future', 'sale', '4.8million'], ['historic', 'attraction', 'pier', 'market', 'owner', 'cuerden', 'leisure', 'collective', 'asking', 'price', '12.6million'], ['blackpool', 'central', 'pier', 'home', 'famous', '33', 'metre', 'high', 'ferris', 'wheel', 'close', 'pier', 'blackpool', 'tower', 'blackpool', 'south', 'pier', 'llandudno', 'pier', 'wale', 'construct', '19th', 'century'], ['scroll', 'video'], ['grab', 'blackpool', 'central', 'pier', 'sale', '5million', 'blackpool', 'south', 'pier'], ['llandudno', '695', 'metre', 'long', 'pier', 'grade', 'list', 'mean', 'future', 'owner', 'require', 'list', 'building', 'consent', 'change', 'structure'], ['blackpool', 'central', 'pier', 'stand', '341', 'metre', 'long', 'blackpool', 'south', 'pier', '150', 'metre', 'long', 'benefit', 'list', 'status', 'despite', 'opening', '1864', '1892', 'respectively'], ['richard', 'baldwin', 'director', 'bilfinger', 'gva', 'sell', 'pier', 'cuerden', 'leisure', 'say', 'unlikely', 'new', 'owner', 'significant', 'change'], ['say', 'order', 'sustain', 'value', 'ensure', 'value', 'appreciate', 'change', 'dramatically', 'new', 'ownership'], ['opinion', 'value', 'exist', 'use', 'unlikely', 'change', 'blackpool', 'pier'], ['victorian', 'pier', 'construct', 'cast', 'iron', 'pile', 'steel', 'frame', 'wooden', 'decking', 'boast', 'amusement', 'arcade', 'ride', 'generate', 'collective', 'income', '1.6', 'million', 'year', 'cuerden', 'leisure', 'annual', 'concession', 'agreement'], ['grade', 'list', 'llandudno', 'pier', 'sale', 'altogether', 'buy', '12.6million'], ['st', 'john', 'stott', 'director', 'cuerden', 'leisure', 'own', 'eastbourne', 'pier', 'say', 'group', 'sell', 'pier', 'restructure', 'asset'], ['say', 'asset', 'jewel', 'crown', 'uk', 'coastline', 'delighted', 'offer', 'market', 'separate', 'lot', 'collectively'], ['mr', 'baldwin', 'add', 'pier', 'truly', 'iconic', 'structure', 'having', 'popular', 'visitor', 'attraction', 'uk', 'good', 'know', 'resort', 'century'], ['pier', 'offer', 'sale', 'freehold', 'subject', 'concession', 'agreement', 'place'], ['profitable', 'attraction', 'confident', 'sale', 'attract', 'major', 'interest'], ['spokesman', 'national', 'pier', 'society', 'promote', 'preservation', 'continue', 'enjoyment', 'seaside', 'pier', 'uk', 'say', 'pier', 'good', 'order', 'trading', 'successfully', 'give', 'easter', 'doubt', 'quickly', 'find', 'buyer', 'buyer']]"""
txt = string_to_list(txt)
a = get_sentence_embeddings(txt)
print(len(a))

18


In [33]:
for name, df in datasets.items():
    df['art_sent_embeddings'] = df['clean_sents_art'].apply(string_to_list).apply(get_sentence_embeddings)
    df['high_sent_embeddings'] = df['clean_sents_high'].apply(string_to_list).apply(get_sentence_embeddings)
    datasets[name] = df

In [22]:
#for name, df in datasets.items():
 #   if 'high_sent_embeddings' in df.columns: df.drop('high_sent_embeddings', axis=1, inplace=True)

In [43]:
# Check columns + results. All ok.
#for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['clean_sents_art'].head(4)
#datasets['validation']['clean_sents_art'].head()
#datasets['test'].head(2)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Cosine Similarity (labels)

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(article_embeddings, highlight_embeddings):
    # Calculate cosine similarity between each article sentence and each highlight sentence.
    similarity_scores = []
    for art_emb in article_embeddings:
        # Scores for each article sentence against all highlight sentences.
        scores = [cosine_similarity([art_emb], [high_emb])[0][0] for high_emb in highlight_embeddings]
        # Aggregate the scores by taking the maximum.
        similarity_scores.append(max(scores) if scores else 0)
    return similarity_scores


for name, df in datasets.items():
    # Calculate similarity scores for each row in the dataframe
    similarity_scores = []
    for index, row in df.iterrows():
        # Use the precomputed sentence embeddings
        article_embeddings = row['art_sent_embeddings']
        highlight_embeddings = row['high_sent_embeddings']

        # Calculate similarity scores
        scores = calculate_similarity(article_embeddings, highlight_embeddings)

        # Store the scores
        similarity_scores.append(scores)
    
    # Add the similarity scores as a new column to the dataframe
    df['similarity_scores'] = similarity_scores
    datasets[name] = df


In [38]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['similarity_scores'].head()
#datasets['validation']['clean_sents_art'].head()
#datasets['test'].head(2)

Columns in train dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores'],
      dtype='object')
Columns in validation dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores'],
      dtype='object')
Columns in test dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores'],
      dtype='object')


0                                                                                                                                                      [0.78614044, 0.6281063, 0.7003851, 0.2905049, 0.8066679, 0.693566, 0.77165043, 0.7447659, 0.4227616, 0.6808056, 0.76678455, 0.64269066, 0.62133586, 0.6361592, 0.65129006]
1    [0.7712006, 0.77382934, 0.63337034, 0.6840277, 0.8246098, 0.7272917, 0.5598484, 0.64864475, 0.67015505, 0.616089, 0.77523786, 0.70468974, 0.66512036, 0.6574998, 0.72593665, 0.6758898, 0.6753683, 0.7291945, 0.5874759, 0.71150357, 0.7354912, 0.66362464, 0.6546197, 0.7050079, 0.68018496, 0.5729586, 0.69163215, 0.6215]
2                                                      [0.8799081, 0.66603494, 0.46760952, 0.5707332, 0.26893198, 0.5221286, 0.5097554, 0.7589295, 0.77468795, 0.54191643, 0.36442742, 0.51211226, 0.83913976, 0.73731077, 0.30618584, 0.7807745, 0.7302368, 0.60418886, 0.49855953, 0.4104001, 0.45465663, 0.6008837, 0.4152067]
3                                 

In [46]:
datasets['train'].head(1)

article  \
0  Since giving birth to her daughter Arabella four months ago, Rebecca Ferguson has become the victim of cruel taunts about her size 12 post-pregnancy body. But the singer has now hit back at her critics and vowed to resist the pressure to slim down. Speaking to Fabulous magazine, the 28-year-old former X Factor star confessed that recent comments aimed at her weight, and questions about when she'll 'drop the post-baby pounds', have left her more determined than ever to stand up for herself. Scroll down for video . Rebecca Ferguson gave birth to gorgeous baby Arabella just four months ago, but has faced constant questions, even from those close to her, about how and when she'll lose her 'baby weight' The singer says even though she's only a size 12 following her third pregnancy, there has been a lot of pressure on her as a celebrity mum to 'snap back into shape' within a few weeks (Left: Rebecca during her third trimester and right: Rebecca in a bikini last month . She said 'There's this culture where we celebrate people snapping back into shape a week after the birth, but I don't want to be one of those people - I just want to enjoy Arabella.' Rebecca, who has never named the father of Arabella publicly, also has two other children called Lillie, aged 10, and Karl, nine, with her former boyfriend and teenage sweetheart Karl Dures, 29. And the singer now admits that she regrets not having had this empowering new mindset after she had her other kids. Describing how she felt as a new mum in previous years, the Get Happy singer said, 'When I look back, I think I should have just enjoyed my babies and not worried about my weight or being skinny. 'I'm curvy, I've got thighs, but I'm not big. I eat healthily and I'm breastfeeding, so I know the weight will come off naturally.' Rebecca pictured with her two older children Lillie 10, (left) and Karl, nine, (right) Currently promoting her new album Lady Sings The Blues, it would be easy for Rebecca to feel the need to slim down for upcoming photo shoots and interviews. But while some celebrities are famed for getting their impossibly taut figures back just weeks after giving birth, Rebecca says she's not feeling that pressure this time around. Instead, she's standing up for new mums everywhere, by taking aim at the critics who try to publicly shame women into losing weight. The former X Factor contestant has refused to name and shame her weight critics, but revealed some have been a lot closer to home than she ever imagined (Pictured left: In concert at the St James Theatre, London, last month, and right: on This Morning last week) She said: 'Women's bodies are amazing, what our bodies can do is incredible so it's sad that we get distracted - all this stuff about being skinny, be this, be that, they're all distractions. 'They rob us of what we should be focusing on and that's sad.' Though the singer has refused to name and shame her own critics, she has revealed that they're a lot closer home than she ever imagined. 'I just want to enjoy Arabella,' said Rebecca, who previously worried about dropping post-baby pounds as quickly as possible (Rebecca pictured with her newborn) Rebecca also spoke of her toughest year yet during the interview, when her partner walked out after discovering she was pregnant with Arabella. She revealed that although it has been hard to live life as a single mother again, she is now moving forward and enjoying her life as a new mum. The star hasn't been left completely on her own though - her ex-partner Karl pops by regularly to help look after the children. And while Rebecca admits that many of their close friends and family would love to see them get back together, and she is open to the idea, she is enjoying being on her own for the time being. Rebecca flaunted her new curves on the red carpet at the Brit Awards last month and says she's proud of her womanly shape .   

                                                                             

In [47]:
def apply_threshold(scores, threshold=0.6):
    return [1 if score >= threshold else 0 for score in scores]

for name, df in datasets.items():
    df['binary_labels'] = df['similarity_scores'].apply(apply_threshold)
    datasets[name] = df


In [48]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['binary_labels'].head()
#datasets['validation'].head()
#datasets['test'].head(2)

Columns in train dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels'],
      dtype='object')
Columns in validation dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels'],
      dtype='object')
Columns in test dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels'],
      dtype='object')


0                                           [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
1    [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
2                   [1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0]
3                                  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1]
4                                  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]
Name: binary_labels, dtype: object

In [61]:
datasets['train']['clean_sents_art'].head(2)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Surface Features: Sentence Position
" The sentences in the earlier parts of a document are more important than
sentences in later parts."

-- p.3, *Extractive Summarization Using Supervised and Semi-Supervised Learning*

In [ ]:
nlp = spacy.load("en_core_web_sm")

def sentence_position_scores(lst):
    total_sentences = len(lst)
    print(total_sentences)
    # Calculate the position score for each sentence.
    sent_position_scores = [1 - (i / (total_sentences - 1)) if total_sentences > 1 else 1 for i in range(total_sentences)]
    return sent_position_scores

# Example
example_lemmatized_sentences =  [['give', 'birth', 'daughter', 'arabella', 'month', 'ago', 'rebecca', 'ferguson', 'victim', 'cruel', 'taunt', 'size', '12', 'post', 'pregnancy', 'body'], ['singer', 'hit', 'critic', 'vow', 'resist', 'pressure', 'slim'], ['speak', 'fabulousmagazine', '28', 'year', 'old', 'x', 'factor', 'star', 'confess', 'recent', 'comment', 'aim', 'weight', 'question', 'drop', 'post', 'baby', 'pound', 'leave', 'determined', 'stand'], ['scroll', 'video'], ['rebecca', 'ferguson', 'give', 'birth', 'gorgeous', 'baby', 'arabella', 'month', 'ago', 'face', 'constant', 'question', 'close', 'lose', 'baby', 'weight', 'singer', 'say', 'size', '12', 'follow', 'pregnancy', 'lot', 'pressure', 'celebrity', 'mum', 'snap', 'shape', 'week', 'karl', 'currently', 'promote', 'new', 'album', 'lady', 'sing', 'blue', 'easy', 'rebecca', 'feel', 'need', 'slim', 'upcoming', 'photo', 'shoot', 'interview'], ['celebrity', 'fame', 'get', 'impossibly', 'taut', 'figure', 'week', 'give', 'birth', 'rebecca', 'say', 'feel', 'pressure', 'time'], ['instead', 'stand', 'new', 'mum', 'take', 'aim', 'critic', 'try', 'publicly', 'shame', 'woman', 'lose', 'weight'], ['x', 'factor', 'contestant', 'refuse', 'shame', 'weight', 'critic', 'reveal', 'lot', 'close', 'home', 'imagine', 'say', 'woman', 'body', 'amazing', 'body', 'incredible', 'sad', 'distract', 'stuff', 'skinny', 'distraction'], ['rob', 'focus', 'sad'], ['singer', 'refuse', 'shame', 'critic', 'reveal', 'lot', 'close', 'home', 'imagine'], ['want', 'enjoy', 'arabella', 'say', 'rebecca', 'previously', 'worry', 'drop', 'post', 'baby', 'pound', 'quickly', 'possible', 'rebecca', 'speak', 'tough', 'year', 'interview', 'partner', 'walk', 'discover', 'pregnant', 'arabella'], ['reveal', 'hard', 'live', 'life', 'single', 'mother', 'move', 'forward', 'enjoy', 'life', 'new', 'mum'], ['star', 'leave', 'completely', 'ex', 'partner', 'karl', 'pop', 'regularly', 'help', 'look', 'child'], ['rebecca', 'admit', 'close', 'friend', 'family', 'love', 'open', 'idea', 'enjoy', 'time'], ['rebecca', 'flaunt', 'new', 'curve', 'red', 'carpet', 'brit', 'award', 'month', 'say', 'proud', 'womanly', 'shape']], [['ireland', 'clung', 'claim', 'controversial', 'run', 'win', 'zimbabwe', 'bid', 'reach', 'world', 'cup', 'quarter', 'final', 'track'], ['ed', 'joyce', 'day', 'international', 'century', 'help', 'ireland', 'post', '331', 'high', 'score', 'world', 'cup', 'zimbabwe', 'look', 'like', 'run', 'record', 'chase', 'debatable', 'john', 'mooney', 'catch'], ['replay', 'appear', 'mooney', 'step', 'rope', 'hold', 'remove', 'sean', 'williams', 'short', 'century', 'seemingly', 'control', 'pursuit'], ['alex', 'cusack', 'ireland', 'congratulate', 'team', 'mate', 'get', 'wicket', 'tawanda', 'mupariwa'], ['kevin', "o'brien", 'celebrate', 'sean', 'williams', 'controversially', 'catch', 'john', 'mooney'], ['ireland', 'celebrate', 'win', 'pulsate', 'world', 'cup', 'clash', 'zimbabwe', 'knock'], ['umpire', 'call', 'judge', 'catch', 'williams', 'remain', 'field', 'play', 'instead', 'opt', 'word', 'mooney', 'take', 'catch', 'inside', 'rope'], ['drama', 'follow', 'number', '10', 'tawanda', 'mupariwa', 'slap', '19', 'penultimate', 'deliver', 'kevin', "o'brien", 'leave', 'zimbabwe', 'need', 'seven', 'ball'], ['alex', 'cusack', 'hold', 'nerve', 'claim', 'final', 'wicket', 'get', 'regis', 'chakabva', 'drag', 'mupariwa', 'ski', 'catch', 'william', 'porterfield', 'gratefully', 'accept'], ['joyce', '112', 'earn', 'man', 'match', 'award', 'cusack', '32', 'invaluable', 'especially', 'remove', 'zimbabwe', 'skipper', 'brendon', 'taylor', '121'], ['ed', 'joyce', 'celebrate', 'make', 'day', 'international', 'century', 'help', 'ireland', '331', '8'], ['joyce', 'score', 'impressive', 'run', 'ireland', 'boost', 'chance', 'world', 'cup', 'qualification'], ['victory', 'ireland', 'second', 'member', 'nation', 'tournament', 'open', 'campaign', 'win', 'west', 'indie'], ['likely', 'need', 'pull', 'shock', 'final', 'pool', 'game', 'india', 'pakistan'], ['pakistan', 'surprise', 'win', 'south', 'africa', 'early', 'day', 'ireland', 'favour', 'clash', 'adelaide', 'march', '15', 'loom', 'potential', 'decider'], ['john', 'mooney', 'celebrates', 'get', 'wicket', 'sikandar', 'raza', 'help', 'way', 'victory'], ['ireland', 'stand', 'national', 'anthem', '2015', 'icc', 'cricket', 'world', 'cup', 'zimbabwe'], ['joyce', 'fourth', 'ireland', 'player', 'score', 'world', 'cup', 'century', 'give', 'help', 'hand', 'sloppy', 'zimbabwe', 'fielding', 'display'], ['sussex', 'batsman', 'twice', 'drop', 'edge', 'ball', 'short', 'slip', 'imperious', 'hit', 'four', 'six'], ['join', 'wicket', 'partnership', '138', 'andy', 'balbirnie', 'run', 'away', 'follow', 'joyce', 'figure', 'run', 'late', 'scramble'], ['ireland', 'take', '108', 'final', '10', 'over', 'reach', 'high', 'odi', 'score', 'surpass', '329', 'seven', 'rack', 'famous', 'world', 'cup', 'win', 'england', 'bangalore', 'year', 'ago'], ['kevin', "o'brien", 'celebrate', 'take', 'controversial', 'wicket', 'zimbabwe', 'batsman', 'sean', 'williams'], ['williams', 'react', 'get', 'bellerive', 'oval', 'ground', 'despite', 'look', 'like', 'mooney', 'touch', 'foam'], ['zimbabwe', 'crash', '74', 'reply', 'ireland', 'total', 'control', 'taylor', 'williams', 'turn', 'momentum', 'match', '149', 'run', 'stand'], ['taylor', 'reach', 'century', '79', 'ball', 'ireland', 'attack', 'look', 'toothless', 'cusack', 'produce', 'slow', 'ball', 'fool', 'zimbabwe', 'skipper', 'spoon', 'catch', 'mid'], ['williams', 'keep', 'scoreboard', 'tick', 'reduce', 'task', 'manageable', '32', '20', 'ball', 'mooney', 'controversial', 'catch', 'claim'], ['critical', 'moment', 'mupariwa', 'ireland', 'sweat', 'hit', 'kevin', "o'brien", 'four', 'cusack', 'clean', 'tail', 'thrilling', 'finale'], ['irish', 'player', 'congratulate', 'final', 'victory', 'african', 'country']]
print(sentence_position_scores(example_lemmatized_sentences))

In [101]:
for name, df in datasets.items():
    df['sent_position'] = df['clean_sents_art'].apply(string_to_list).apply(sentence_position_scores)
    datasets[name] = df

15
28
23
18
18
77
29
32
27
28
36
74
10
27
18
33
45
12
75
12
64
28
23
10
173
46
25
29
72
20
21
41
66
14
87
38
46
29
21
30
63
33
19
18
10
10
22
12
17
62
14
11
16
77
35
39
33
54
25
26
12
21
35
28
22
15
16
31
30
24
31
17
15
42
16
84
27
27
15
40
47
25
57
16
36
43
18
42
32
22
7
38
36
45
16
53
33
30
36
13
17
49
48
53
26
73
42
51
20
12
30
36
48
23
16
22
57
16
15
39
52
22
24
24
47
15
24
28
16
38
52
39
82
45
15
28
23
18
18
77
29
32
27
28
36
74
10
27
18
33
45
12
75
12
64
28
23
10
173
46
25
29
72
20
21
41
66
14
87
38
46
29
21
30
63
33
19
18
10
10
22
12
17
62
14
11
16
77
35
39
33
54
25
26
12
21
35
28
22
15
16
31
30
24
31
17
15
42
16
84
27
27
15
40
47
25
57
16
36
43
18
42
32
22
7
38
36
45
16
53
33
30
36
13
17
49
48
53
26
73
42
51
20
12
30
36
48
23
16
22
57
16
15
39
52
22
24
24
47
15
24
28
16
38
52
39
82
45
28
70
20
68
51
42
43
20
19
57
40
14
24
22
38
13
25
32
41
69
17
87
25
91
52
38
12
23
10
23
13
41
24
22
14
23
22
39
24
14
15
37
84
74
24
18
20
21
38
46
15
28
27
20
44
32
20
73
23
36
19
56
17
42
28
7

In [65]:
#for name, df in datasets.items():
    #if 'sent_position' in df.columns: df.drop('sent_position', axis=1, inplace=True)

In [67]:
pd.set_option("display.max_columns", None) # show all cols
pd.set_option('display.max_colwidth', None) # show full width of showing cols
pd.set_option('display.max_rows', None)

## Surface Features: Sentence Length

"A sentence is important if the number of words (except stop words) in it is within a certain range."

-- p.3, *Extractive Summarization Using Supervised and Semi-Supervised Learning*

In [98]:
# Sentence length can provide insights into the complexity and information density of a sentence, which might correlate with its summary-worthiness.
# In general, both very short and very long sentences might be less likely to be included in a summary.

def sentence_length_on_clean(lemmatized_sentences):
    # Initialize an empty list to store sentence lengths
    sentence_lengths = []

    # Iterate over each sentence (which is a list of words)
    for sentence in lemmatized_sentences:
        # Count the number of words in the sentence
        length = len(sentence)

        # Append the count to the sentence_lengths list
        sentence_lengths.append(length)

    return sentence_lengths

# Example usage
example_lemmatized_sentences = [['give', 'birth', 'daughter', 'arabella', 'month', 'ago', 'rebecca', 'ferguson', 'victim', 'cruel', 'taunt', 'size', '12', 'post', 'pregnancy', 'body'], ['singer', 'hit', 'critic', 'vow', 'resist', 'pressure', 'slim']]
print(sentence_length_on_clean(example_lemmatized_sentences))


[16, 7]


In [99]:
for name, df in datasets.items():
    df['sent_length'] = df['clean_sents_art'].apply(string_to_list).apply(sentence_length_on_clean)
    datasets[name] = df

In [102]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
#datasets['train']['sent_length'].head()
#datasets['validation']['sent_length'].head()
#datasets['test'].head(2)

Columns in train dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent', 'sent_length', 'sent_position'],
      dtype='object')
Columns in validation dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent', 'sent_length', 'sent_position'],
      dtype='object')
Columns in test dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high

In [78]:
# Print the first few entries of the 'clean_sents_art' column
for name, df in datasets.items():
    print(f"Dataset: {name}")
    print(df['clean_sents_art'].head(2))


Dataset: train
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [83]:
#for name, df in datasets.items():
 #   if 'sent_length' in df.columns: df.drop('sent_length', axis=1, inplace=True)

## Content Features: Signature terms / Named Enitites (counts)
Using the count of NEs in each sentence as a feature can provide a simple yet effective way to measure the richness and importance of the sentence. Sentences containing significant named entities (such as important people, organizations, locations, etc.) are more likely to be essential to the overall narrative or argument of the article, making them strong candidates for inclusion in summaries. For example, a sentence with a higher number of named entities (NEs) may be more informative or central for the article's main theme, and more likely to be summary-worthy.

In [89]:
# Counted only from the 'article' columns before preprocessing, as NE extraction works best in unprocessed text.
# NEs provide valuable insights into the key topics and important elements of the articles.

def count_named_entities_per_sentence(text):
    doc = nlp(text)

    ne_counts_per_sentence = []

    for sent in doc.sents:
        ne_counts_per_sentence.append(len(sent.ents))
    return ne_counts_per_sentence


for name, df in datasets.items():
    df['NE_count'] = df['boiler-free_art'].apply(count_named_entities_per_sentence)
    datasets[name] = df

In [104]:
# Check columns + results. All ok.
#for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['NE_count'].head()
#datasets['validation']['clean_sents_art'].head()
#datasets['test'].head(2)

0                                     [4, 0, 2, 0, 3, 7, 2, 0, 1, 0, 0, 0, 5, 0, 1, 1, 3]
1    [4, 9, 3, 3, 3, 3, 3, 8, 6, 8, 5, 3, 3, 4, 6, 2, 4, 5, 4, 6, 9, 3, 3, 7, 5, 4, 5, 2]
2                [4, 2, 0, 1, 1, 0, 2, 2, 3, 3, 4, 1, 2, 4, 2, 2, 5, 6, 3, 2, 3, 1, 1, 1]
3                               [2, 4, 7, 0, 0, 3, 4, 6, 2, 0, 0, 4, 3, 5, 1, 4, 0, 0, 3]
4                                  [3, 5, 2, 0, 3, 4, 3, 3, 0, 3, 4, 3, 2, 4, 4, 5, 4, 2]
Name: NE_count, dtype: object

In [94]:
# Extract the NEs themselves, for inspection.

def named_ents_per_sentence(text):
    doc = nlp(text)
    
    named_entities_per_sentence = []
    
    # Iterate over each sentence
    for sent in doc.sents:
        ents = [ent.text for ent in sent.ents]
        named_entities_per_sentence.append(ents)
    return named_entities_per_sentence

for name, df in datasets.items():
    df['NEs_per_sent'] = df['boiler-free_art'].apply(named_ents_per_sentence)
    datasets[name] = df


In [96]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['NEs_per_sent'].head(2)
#datasets['validation'].head(2)
#datasets['test'].head(2)

Columns in train dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent'],
      dtype='object')
Columns in validation dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent'],
      dtype='object')
Columns in test dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      [[Arabella, four months ago, Rebecca Ferguson, 12

## Content Features: High Frequency Words ???

## Event Features: Verbs (counts)
Verbs are integral to conveying actions and events within a text. In news articles, for instance, verbs play a crucial role in reporting what happened, when, and to whom. A higher frequency of verbs in a sentence might indicate that it is action-packed or event-rich, which could be a characteristic of summary-worthy content.

In [112]:
# Counted only from the ''boiler-free_art'' columns before preprocessing, as verb (and, in general, part-of-speech) extraction works best on unprocessed text.

def count_verbs_per_sentence(text):
    doc = nlp(text)
    verb_counts = [sum(1 for token in sent if token.pos_ == 'VERB') for sent in doc.sents]
    return verb_counts

for name, df in datasets.items():
    df['VERB_count'] = df['boiler-free_art'].apply(count_verbs_per_sentence)
    datasets[name] = df

In [109]:
#for name, df in datasets.items():
 #   if 'VERB_count' in df.columns: df.drop('VERB_count', axis=1, inplace=True)

In [113]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['VERB_count'].head(2)
#datasets['validation'].head(6)
#datasets['test'].head(2)

Columns in train dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent', 'sent_length', 'sent_position',
       'VERB_count'],
      dtype='object')
Columns in validation dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent', 'sent_length', 'sent_position',
       'VERB_count'],
      dtype='object')
Columns in test dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_

0                                     [2, 4, 6, 1, 3, 6, 5, 5, 5, 3, 2, 5, 8, 4, 3, 5, 2]
1    [3, 4, 5, 2, 2, 4, 6, 6, 6, 2, 3, 2, 1, 2, 1, 3, 1, 4, 3, 3, 4, 2, 3, 2, 5, 4, 3, 1]
Name: VERB_count, dtype: object

In [116]:
# Extract the VERBs themselves, for inspection.

def verbs_per_sentence(text):
    doc = nlp(text)
    verbs_by_sentence = [[token.text for token in sent if token.pos_ == 'VERB'] for sent in doc.sents]
    return verbs_by_sentence

for name, df in datasets.items():
    df['VERBS'] = df['boiler-free_art'].apply(verbs_per_sentence)
    datasets[name] = df


In [117]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['VERBS'].head(2)
#datasets['validation'].head(2)
#datasets['test'].head(2)

Columns in train dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent', 'sent_length', 'sent_position',
       'VERB_count', 'VERBS'],
      dtype='object')
Columns in validation dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent', 'sent_length', 'sent_position',
       'VERB_count', 'VERBS'],
      dtype='object')
Columns in test dataset: Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_a

0                                                                                                                                                                                                                            [[giving, become], [hit, vowed, resist, slim], [Speaking, confessed, aimed, drop, left, stand], [Scroll], [gave, faced, lose], [says, following, snap, promoting, feel, slim], [famed, getting, giving, says, feeling], [standing, taking, try, shame, losing], [refused, name, shame, revealed, imagined], [said, get, distracted], [rob, focusing], [refused, name, shame, revealed, imagined], [want, enjoy, said, worried, dropping, spoke, walked, discovering], [revealed, live, moving, enjoying], [left, help, look], [admits, love, see, get, enjoying], [flaunted, says]]
1    [[claim, keep, reach], [helped, post, looked, running], [appeared, show, stepped, held, remove], [congratulated, getting], [celebrates, caught], [celebrate, winning, pulsating, knocking], [called, judge, remain

In [ ]:
# thematic word: the top n words with the highest frequency in the cleaned text article.
# It shows that eight out of 10 of the top 10 words are contained in the summary while decrease harshly after the top 10 words. Therefore, the thematic
 # words were set as the top 10 words in the thematic word feature extraction
 # THE TOP 10 WORDS WITH THE HIGHEST FREQUENCY IN THE CLEANED TEXT ORDERED IN DESCDENDING ORDER. NAI
 # FIND TOP 10 verbs FOR THIS ARTICLE


In [118]:
# check how many columns created so far. All ok.
for name, df in datasets.items():
    print(f"THe number of columns in the {name} dataset is", len(df.columns), "and these are: ", df.columns)

THe number of columns in the train dataset is 19 and these are:  Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent', 'sent_length', 'sent_position',
       'VERB_count', 'VERBS'],
      dtype='object')
THe number of columns in the validation dataset is 19 and these are:  Index(['article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'clean_sents_art', 'clean_sents_high', 'art_sent_embeddings',
       'high_sent_embeddings', 'similarity_scores', 'binary_labels',
       'NE_count', 'NEs_per_sent', 'sent_length', 'sent_position',
       'VERB_count', 'VERBS'],
      dtype='object')
THe number of columns in the test dataset is 19 and these are:  Index(['article', 'highlights',

## Create the X_train input 

In [121]:
# Construct a new DataFrame where each row represents a sentence along with its features and label.
# The columns might include sentence text, sentence embeddings, verb count, named entity count, sentence position, and a binary label indicating whether the sentence is in the summary.

# Initialize an empty dictionary to store the X_train datasets for each dataframe.
X_train_datasets = {}

for name, df in datasets.items():
    X_train = []

    for index, row in df.iterrows():
        # Assuming each of these columns contains a list of features for each sentence.
        sentence_embeddings = row['art_sent_embeddings']
        position_scores = row['sent_position']
        sentence_lengths = row['sent_length']
        ne_counts = row['NE_count']
        verb_counts = row['VERB_count']

        if all(len(lst) == len(sentence_embeddings) for lst in [position_scores, sentence_lengths, ne_counts, verb_counts]):
            article_features = []
            for i in range(len(sentence_embeddings)):
                # Convert the list of additional features to a NumPy array.
                additional_features = np.array([position_scores[i], sentence_lengths[i], ne_counts[i], verb_counts[i]])
                # Concatenate using NumPy's concatenate function.
                sentence_feature = np.concatenate([sentence_embeddings[i], additional_features])
                article_features.append(sentence_feature)
            X_train.append(article_features)
        else:
            print(f"Feature length mismatch in row {index} of {name} dataset")

    # Store the X_train for this dataframe in the dictionary
    X_train_datasets[name] = X_train

# Now X_train_datasets contains the combined features for each dataframe in the datasets dictionary.

Feature length mismatch in row 0 of train dataset
Feature length mismatch in row 2 of train dataset
Feature length mismatch in row 3 of train dataset
Feature length mismatch in row 5 of train dataset
Feature length mismatch in row 7 of train dataset
Feature length mismatch in row 8 of train dataset
Feature length mismatch in row 9 of train dataset
Feature length mismatch in row 10 of train dataset
Feature length mismatch in row 11 of train dataset
Feature length mismatch in row 13 of train dataset
Feature length mismatch in row 14 of train dataset
Feature length mismatch in row 16 of train dataset
Feature length mismatch in row 18 of train dataset
Feature length mismatch in row 20 of train dataset
Feature length mismatch in row 21 of train dataset
Feature length mismatch in row 24 of train dataset
Feature length mismatch in row 25 of train dataset
Feature length mismatch in row 28 of train dataset
Feature length mismatch in row 29 of train dataset
Feature length mismatch in row 30 of t

## Create y_train labels

In [122]:
y_train_datasets = {}

for name, df in datasets.items():
    # Extract the 'binary_labels' column as the labels for training
    y_train = df['binary_labels'].tolist()
    y_train_datasets[name] = y_train

In [123]:
# Check the first few labels of each dataset.
for name, labels in y_train_datasets.items():
    print(f"Dataset: {name}")
    print(labels[:5])


Dataset: train
[[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1], [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]]
Dataset: validation
[[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1], [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]]
Dataset: test
[[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1,